<a href="https://colab.research.google.com/github/studentAybu/ML_GlobalCountryMetrics2025/blob/main/ML_GlobalCountryMetrics2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install opendatasets

In [35]:
import opendatasets as od

od.download(
    "https://www.kaggle.com/datasets/prashantdhanuk/global-country-metrics-2025-hdi-gdp-pop-area?resource=download")

Skipping, found downloaded files in "./global-country-metrics-2025-hdi-gdp-pop-area" (use force=True to force download)


In [36]:
import pandas as pd

# reading the file
file =('/content/global-country-metrics-2025-hdi-gdp-pop-area/countries_metric - Sheet1.csv')
newData = pd.read_csv(file)

# displaying the contents of the file
newData.head()

,country_name,Population (in millions),Nominal Gross Domestic Product (in USD),Nominal GDP Per capita (in USD),GDP Per capita PPP (in USD),Human Development Index (HDI),GINI,AREA (in Sq km)
0,Afghanistan,"41,454,761",$17.329 billion,411,"2,174",0.496,35.2,"652,867"
1,Albania,"2,402,113",$28.372 billion,"10,526","23,404",0.810,33.0,"28,748"
2,Algeria,"46,700,000",$266.78 billion,"5,722","16,483",0.763,27.6,"2,381,741"
3,Andorra,"87,486",$3.897 billion,"44,900","69,146",0.913,40.0,467.63
4,Angola,"39,040,039",$93.796 billion,"2,432",7077,0.616,51.3,"1,246,700"


In [37]:
import pandas as pd
data=newData
#total
total_rows = data.shape[0]
missing_rows = data.isnull().sum().sum()
missing_rows_percent = round((missing_rows / total_rows) * 100, 2)


# Missing Value Count and Percentage by Column
missing_data = newData.isnull().sum()
missing_percent = (missing_data / total_rows * 100).round(2)

# Create a DataFrame with missing value counts and percentages by column
missing_report = pd.DataFrame({
    'Missing Value Count': missing_data,
    'Missing Value Percentage (%)': missing_percent
})

# Create a single-row DataFrame summarizing total missing data by rows
summary_row = pd.DataFrame({
    'Missing Value Count': [missing_rows],
    'Missing Value Percentage (%)': [missing_rows_percent]
}, index=['Total missing data'])

final_report = pd.concat([summary_row, missing_report])

print("🧾 Missing Data Report:")
print(final_report)


🧾 Missing Data Report:
                                         Missing Value Count  \
Total missing data                                        22   
country_name                                               0   
Population (in millions)                                   0   
Nominal Gross Domestic Product (in USD)                    0   
Nominal GDP Per capita (in USD)                            0   
GDP Per capita PPP (in USD)                                0   
Human Development Index (HDI)                              2   
GINI                                                      20   
AREA (in Sq km)                                            0   

                                         Missing Value Percentage (%)  
Total missing data                                              11.34  
country_name                                                     0.00  
Population (in millions)                                         0.00  
Nominal Gross Domestic Product (in USD)         

In [38]:
newData = data.dropna()

In [40]:
import pandas as pd

#total
total_rows = newData.shape[0]
missing_rows = newData.isnull().sum().sum()
missing_rows_percent = round((missing_rows / total_rows) * 100, 2)


# Missing Value Count and Percentage by Column
missing_data = newData.isnull().sum()
missing_percent = (missing_data / total_rows * 100).round(2)

# Create a DataFrame with missing value counts and percentages by column
missing_report = pd.DataFrame({
    'Missing Value Count': missing_data,
    'Missing Value Percentage (%)': missing_percent
})

# Create a single-row DataFrame summarizing total missing data by rows
summary_row = pd.DataFrame({
    'Missing Value Count': [missing_rows],
    'Missing Value Percentage (%)': [missing_rows_percent]
}, index=['Total missing data'])

final_report = pd.concat([summary_row, missing_report])

print("🧾 Missing Data Report:")
print(final_report)


🧾 Missing Data Report:
                                         Missing Value Count  \
Total missing data                                         0   
country_name                                               0   
Population (in millions)                                   0   
Nominal Gross Domestic Product (in USD)                    0   
Nominal GDP Per capita (in USD)                            0   
GDP Per capita PPP (in USD)                                0   
Human Development Index (HDI)                              0   
GINI                                                       0   
AREA (in Sq km)                                            0   

                                         Missing Value Percentage (%)  
Total missing data                                                0.0  
country_name                                                      0.0  
Population (in millions)                                          0.0  
Nominal Gross Domestic Product (in USD)         

In [47]:
newData.info()
print(newData.dtypes)


<class 'pandas.core.frame.DataFrame'>
Index: 174 entries, 0 to 193
Data columns (total 12 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   country_name                             174 non-null    object 
 1   Population (in millions)                 174 non-null    Int64  
 2   Nominal Gross Domestic Product (in USD)  174 non-null    object 
 3   Nominal GDP Per capita (in USD)          174 non-null    float64
 4   GDP Per capita PPP (in USD)              174 non-null    float64
 5   Human Development Index (HDI)            174 non-null    float64
 6   GINI                                     174 non-null    float64
 7   AREA (in Sq km)                          174 non-null    float64
 8   Population_million                       174 non-null    float64
 9   HDI_category                             174 non-null    object 
 10  population_density                       174 non-null  

In [48]:
# Function to create HDI category
def categorize_hdi(hdi):
    if pd.isna(hdi):
        return pd.NA
    elif hdi <= 0.549:
        return 'Low'
    elif hdi <= 0.699:
        return 'Medium'
    elif hdi <= 0.799:
        return 'High'
    else:
        return 'Very High'

# Add HDI category column
newData['HDI_category'] = newData['Human Development Index (HDI)'].apply(categorize_hdi)

# Population density (population as integer, area in km2)
newData['population_density'] = (newData['Population (in millions)'] * 1_000_000) / newData['AREA (in Sq km)']

# Total GDP (Nominal GDP Per capita * population)
newData['total_gdp'] = newData['Nominal GDP Per capita (in USD)'] * (newData['Population (in millions)'] * 1_000_000)

print(newData[['population_density', 'total_gdp', 'HDI_category']].head())
print(newData['HDI_category'].value_counts(dropna=False))


   population_density             total_gdp HDI_category
0     63496487.033347   17037906771000000.0          Low
1     83557569.222207   25284641438000000.0    Very High
2     19607505.601995  267217400000000000.0         High
3    187083805.572782    3928121400000000.0    Very High
4     31314702.013315   94945374848000000.0       Medium
HDI_category
Very High    65
High         43
Medium       40
Low          26
Name: count, dtype: int64


In [51]:
import pandas as pd

def clean_numeric_column(df, column_name, remove_chars=None):
    """
    Cleans the specified column and converts it to numeric type (float64).
    remove_chars: list of characters to remove (e.g., [',', '$'])
    """
    df = df.copy()
    series = df[column_name].astype(str)

    if remove_chars:
        for ch in remove_chars:
            series = series.str.replace(ch, '', regex=True)

    series = series.str.strip()

    df[column_name] = pd.to_numeric(series, errors='coerce').astype(float)

    nan_count = df[column_name].isna().sum()
    print(f"Number of NaNs in '{column_name}': {nan_count}")
    print(df[column_name].head())
    return df


def clean_population_column(df, column_name):
    """
    Cleans the population column, converts it to numeric type (Int64),
    and adds a new column for population in millions (float).
    """
    df = df.copy()
    series = df[column_name].astype(str).str.replace(',', '', regex=True).str.strip()
    df[column_name] = pd.to_numeric(series, errors='coerce').astype('Int64')

    nan_count = df[column_name].isna().sum()
    print(f"Number of NaNs in '{column_name}': {nan_count}")

    df['Population_million'] = df[column_name].astype(float) / 1_000_000
    print(df[[column_name, 'Population_million']].head())
    return df


newData = clean_population_column(newData, 'Population (in millions)')

newData = clean_numeric_column(newData, 'Nominal GDP Per capita (in USD)', remove_chars=[',', '$'])
newData = clean_numeric_column(newData, 'GDP Per capita PPP (in USD)', remove_chars=[',', '$'])
newData = clean_numeric_column(newData, 'AREA (in Sq km)', remove_chars=[',', '$'])

print(newData['Nominal GDP Per capita (in USD)'].dtype)
print(newData['GDP Per capita PPP (in USD)'].dtype)
print(newData['AREA (in Sq km)'].dtype)

for col in ['Nominal GDP Per capita (in USD)', 'GDP Per capita PPP (in USD)', 'AREA (in Sq km)']:
    non_numeric = newData[~newData[col].apply(lambda x: isinstance(x, (int, float)))]
    if not non_numeric.empty:
        print(f"Non-numeric values in column '{col}':")
        print(non_numeric[[col]].head())


Number of NaNs in 'Population (in millions)': 0
   Population (in millions)  Population_million
0                  41454761           41.454761
1                   2402113            2.402113
2                  46700000           46.700000
3                     87486            0.087486
4                  39040039           39.040039
Number of NaNs in 'Nominal GDP Per capita (in USD)': 0
0      411.0
1    10526.0
2     5722.0
3    44900.0
4     2432.0
Name: Nominal GDP Per capita (in USD), dtype: float64
Number of NaNs in 'GDP Per capita PPP (in USD)': 0
0     2174.0
1    23404.0
2    16483.0
3    69146.0
4     7077.0
Name: GDP Per capita PPP (in USD), dtype: float64
Number of NaNs in 'AREA (in Sq km)': 0
0     652867.00
1      28748.00
2    2381741.00
3        467.63
4    1246700.00
Name: AREA (in Sq km), dtype: float64
float64
float64
float64


In [52]:
print(newData.dtypes)

country_name                                object
Population (in millions)                     Int64
Nominal Gross Domestic Product (in USD)     object
Nominal GDP Per capita (in USD)            float64
GDP Per capita PPP (in USD)                float64
Human Development Index (HDI)              float64
GINI                                       float64
AREA (in Sq km)                            float64
Population_million                         float64
HDI_category                                object
population_density                         Float64
total_gdp                                  Float64
dtype: object
